<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" width="150px" height="90px" />

## Microestructuras y sistemas de trading

### Laboratorio 4. Análsis Fundamental.

>##### Andrés García
>##### Juan López
>##### Antonio Tirado
>##### Oscar Villa


In [27]:
#Importación del Librerías
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py

In [2]:
cal=pd.read_csv('calendario_economico.csv', header=0, sep=',', index_col=None, parse_dates=False, skip_blank_lines=True)
cal['timestamp'] = pd.to_datetime(cal['timestamp'])
cal['timestamp'] = cal['timestamp'].dt.tz_localize('UTC')
   
precios=pd.read_csv('precios_historicos_eurusd.csv', header=0, sep=',', index_col=None, parse_dates=False, skip_blank_lines=True)
precios['timestamp'] = pd.to_datetime(precios['timestamp'])
precios['timestamp'] = precios['timestamp'].dt.tz_localize('UTC')

In [34]:


def analisis(evento):
    
    #Filtrar el evento en el Calendario Economico
    tb  = cal[(cal['Name'] == evento) & (cal['Currency'] == 'USD')].reset_index(drop=True).dropna(subset=['actual']).fillna(method='ffill',axis=1)

    #Formar los cuatro grupos; A,B,C,D clasificandolas en el criterio:
    # A Actual >= Consensus >= Previous
    # B Actual >= Consensus < Previous
    # C Actual < Consensus >= Previous
    # D Actual < Consensus < Previous    
    condiciones = [(tb.actual>=tb.consensus) & (tb.consensus>=tb.previous),(tb.actual>=tb.consensus) & (tb.consensus<tb.previous)
                  ,(tb.actual<tb.consensus)&(tb.consensus>=tb.previous),(tb.actual<tb.consensus)&(tb.consensus<tb.previous)]
    elecciones =['a','b','c','d']
    clasification=np.select(condiciones,elecciones)
    tb['clasificacion']=clasification 
    
    #Elimina las fechas del calendario que no esten en el dataFrame de precios.
    tb=tb[tb['timestamp'].isin(list(precios['timestamp']))]
    
    #Crear ventanas de tiempo de media hora antes y media hora después de cada evento del calendario economico.   
    ventanas_tb={}
    for i in range(len(tb)):
        ind=int(precios[precios['timestamp']==tb.iloc[i,0]].index.values.astype(int))
        ventana = precios.iloc[ind-6:ind+7].reset_index(drop=True)
        ventanas_tb[tb.iloc[i,0]]=ventana 
        
    #Calcular las 4 metricas y agregarlos a los Data Frames:
    # (Dirección) Close (t_30) - Open(t_0)
    # (Pips Alcistas) High(t_0 : t_30) – Open(t_0)
    # (Pips Bajistas) Open(t_0) – Low(t_0 : t_30)
    # (Volatilidad) High(t_-30 : t_30) , - mínimo low (t_-30:t_30)    
    d  = []
    pa = []
    pb = []
    v  = []

    for i in range(len(tb)): 
        temp     = ventanas_tb[tb.iloc[i,0]]
        dtemp    = temp.iloc[-1,4] - temp.iloc[6,1]
        pipatemp = max(temp.iloc[6:-1,2]) - temp.iloc[6,1]
        pipbtemp = temp.iloc[6,1] - min(temp.iloc[6:-1,3])
        voltemp  = max(temp.iloc[:,2]) - min(temp.iloc[:,3])

        d.append(dtemp)
        pa.append(pipatemp)
        pb.append(pipbtemp)
        v.append(voltemp)

    tb['d']  = d
    tb['pa'] = pa
    tb['pb'] = pb
    tb['v']  = v

    #codigo para graficar las clasificaciones en barras
    fig = go.Figure()
    fig = make_subplots()
    fig.add_trace(go.Bar(x=list(tb['clasificacion'].value_counts().sort_index().index), 
                         y=tb['clasificacion'].value_counts().sort_index(),name='Conteo',
                             hovertemplate = '<i>Casos</i>: %{y}'
                            '<br><b>Identificador</b>: %{x}<br>'),secondary_y=False)
    # Título del grafico
    fig.update_layout(title_text='Escenarios')

    # Titulo eje x
    fig.update_xaxes(title_text='Escenarios')

    # Titulo eje y
    fig.update_yaxes(title_text='<b>Conteo</b> de escenarios', secondary_y=False)

    grafica=fig    

    return tb, grafica

In [35]:
trade_balance, grafica = analisis(evento='Trade Balance')
trade_balance.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,clasificacion,d,pa,pb,v
0,2009-01-13 13:30:00+00:00,USD,Trade Balance,United States,3,-41.58,-56.69,-51.4,a,-0.00337,0.00325,0.00629,0.00964
1,2009-02-11 13:30:00+00:00,USD,Trade Balance,United States,3,-39.9,-41.58,-35.9,c,0.00214,0.00516,0.00028,0.00814
2,2009-03-13 12:30:00+00:00,USD,Trade Balance,United States,3,-36.2,-39.9,-38,a,-0.00141,0.00203,0.00105,0.00600
3,2009-04-09 12:30:00+00:00,USD,Trade Balance,United States,3,-26.13,-36.2,-36.7,b,-0.00329,0.00108,0.00214,0.00559
4,2009-05-12 12:30:00+00:00,USD,Trade Balance,United States,3,-28.53,-26.13,-28.9,b,-0.00310,0.00250,0.00300,0.00648


In [36]:
grafica.show()

In [37]:
ADP_Employment_Change, grafica = analisis(evento='ADP Employment Change')
ADP_Employment_Change.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,clasificacion,d,pa,pb,v
0,2009-01-07 13:15:00+00:00,USD,ADP Employment Change,United States,3,-659,-476,-450,c,0.01126,0.01445,0.00010,0.01573
1,2009-02-04 13:15:00+00:00,USD,ADP Employment Change,United States,3,-522,-659,-525,a,0.00386,0.00627,0.00016,0.00643
2,2009-03-04 13:15:00+00:00,USD,ADP Employment Change,United States,2,-697,-522,-565,d,-0.00044,0.00134,0.00216,0.00350
4,2009-05-06 12:15:00+00:00,USD,ADP Employment Change,United States,2,-491,-708,-644,a,0.00405,0.00461,0.00013,0.00788
5,2009-06-03 12:15:00+00:00,USD,ADP Employment Change,United States,2,-485,-491,-533,b,-0.00210,0.00136,0.00197,0.00486


In [38]:
grafica.show()